In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
import seaborn as sns
from tensorflow import keras
from sklearn.preprocessing import StandardScaler
from scipy import optimize as opt

In [2]:
matches = pd.read_csv("upcoming.csv", index_col=0)

## Cleaning and getting data ready to use.

In [3]:
#Encoding label we make result being a numeric value, which we can predict later. Instead of W, D, L.
yle = LabelEncoder()
matches['result'] = yle.fit_transform(matches['result'])

In [4]:
matches.head()

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,referee,match report,notes,sh,sot,dist,g-xg,npxg/sh,season,team
1,2022-08-14,22:00,La Liga,Matchweek 1,Sun,Away,2,2.0,1.0,Almería,...,Juan Martínez,Match Report,NaN,29.0,15.0,17.9,-0.4,0.09,2023,Real Madrid
2,2022-08-20,22:00,La Liga,Matchweek 2,Sat,Away,2,4.0,1.0,Celta Vigo,...,Jesús Gil,Match Report,NaN,15.0,6.0,18.7,0.6,0.13,2023,Real Madrid
3,2022-08-28,22:00,La Liga,Matchweek 3,Sun,Away,2,3.0,1.0,Espanyol,...,Mario Melero,Match Report,NaN,19.0,6.0,15.6,1.4,0.08,2023,Real Madrid
4,2022-09-03,16:15,La Liga,Matchweek 4,Sat,Home,2,2.0,1.0,Betis,...,José Sánchez,Match Report,NaN,22.0,5.0,17.2,-0.6,0.12,2023,Real Madrid
6,2022-09-11,14:00,La Liga,Matchweek 5,Sun,Home,2,4.0,1.0,Mallorca,...,Jorge Figueroa,Match Report,NaN,22.0,7.0,21.5,2.4,0.07,2023,Real Madrid


In [5]:
#Using pandas sweets.
matches["date"] = pd.to_datetime(matches["date"])

#Creating numeric values for Home and Away, so that machine can learn from it.
matches['venue_c'] = matches['venue'].astype('category').cat.codes

#Creating unique code for each opponent squad.
matches['opp_c'] = matches['opponent'].astype('category').cat.codes

#Creating a normalized hour time.
matches['hour'] = matches['time'].str.replace(":.+", "", regex=True).astype("int")

#Monday = 0, Tuesday = 1, etc.
matches['day_c'] = matches['date'].dt.dayofweek

matches = matches.drop(columns=['round', 'day', 'comp'])

In [6]:
matches = matches[['date', 'result', 'venue', 'gf', 'ga', 'xg', 'xga',
       'poss', 'sh', 'sot', 'dist', 'g-xg', 'team', 'opponent', 'opp_c']]

#Making all teams names correct.
matches = matches.replace("Atlético Madrid", "Atletico Madrid")
matches = matches.replace("Alavés", "Alaves")
matches = matches.replace("Leganés", "Leganes")
matches = matches.replace("Málaga", "Malaga")
matches = matches.replace("La Coruña", "Deportivo La Coruna")
matches = matches.replace("Atlético Madrid", "Atletico Madrid")
matches = matches.replace("Real Betis", "Betis")


In [7]:
is_home = matches['venue'] == "Home"
home = matches[is_home]
away = matches[~is_home]

In [8]:
#Setting up our data, so we can add opponents value at the current time to our DataFrame.
home = home.merge(
    away[["date", "opponent", "team", "xg", "xga", "sh", "poss", "g-xg"]],
    left_on=["date", "team", "opponent"],
    right_on=["date", "opponent", "team"],
    how="outer",
    suffixes=("", "_opp"),
).drop(columns=["opponent_opp", "team_opp"])

In [9]:
home = home.dropna()

In [10]:
matches = home

In [11]:
matches.head()

,date,result,venue,gf,ga,xg,xga,poss,sh,sot,dist,g-xg,team,opponent,opp_c,xg_opp,xga_opp,sh_opp,poss_opp,g-xg_opp
0,2022-09-03,2.0,Home,2.0,1.0,2.6,0.5,49.0,22.0,5.0,17.2,-0.6,Real Madrid,Betis,5.0,0.5,2.6,8.0,51.0,0.5
1,2022-09-11,2.0,Home,4.0,1.0,1.6,0.4,68.0,22.0,7.0,21.5,2.4,Real Madrid,Mallorca,19.0,0.4,1.6,5.0,32.0,0.6
2,2022-10-02,2.0,Home,4.0,1.0,1.6,0.4,68.0,22.0,7.0,21.5,2.4,Real Madrid,Osasuna,21.0,2.1,0.3,15.0,39.0,-1.1
3,2022-10-16,2.0,Home,4.0,1.0,1.6,0.4,68.0,22.0,7.0,21.5,2.4,Real Madrid,Barcelona,4.0,3.0,0.3,16.0,69.0,1.0
4,2022-10-23,2.0,Home,4.0,1.0,1.6,0.4,68.0,22.0,7.0,21.5,2.4,Real Madrid,Sevilla,25.0,1.8,2.4,11.0,41.0,1.2


In [12]:
grouped_matches = matches.groupby('team')

In [13]:
group = grouped_matches.get_group("Real Madrid")

In [14]:
#Short function to get rolling averages as our predictors from last 3 matches.
def rolling_averages(group, cols, new_cols):
    group = group.sort_values('date')
    rolling_stats = group[cols].rolling(3, closed='left').mean()
    group[new_cols] = rolling_stats
    group = group.dropna(subset=new_cols)
    return group

In [15]:
#cols = ['poss', 'sh', 'xg', 'xga', 'g-xg', 'poss_opp', 'sh_opp', 'xg_opp', 'xga_opp', 'g-xg_opp']
cols = ['xg', 'g-xg', 'xg_opp', 'g-xg_opp']
new_cols = [f"{c}R" for c in cols]

In [16]:
#Getting the rolling averages.
matches_rolling = matches.groupby('team').apply(lambda x: rolling_averages(x, cols, new_cols))
matches_rolling = matches_rolling.droplevel('team')
matches_rolling.index = range(matches_rolling.shape[0])

In [17]:
matches_rolling.head()

,date,result,venue,gf,ga,xg,xga,poss,sh,sot,...,opp_c,xg_opp,xga_opp,sh_opp,poss_opp,g-xg_opp,xgR,g-xgR,xg_oppR,g-xg_oppR
0,2017-10-14,1.0,Home,0.0,2.0,0.5,0.7,27.0,8.0,0.0,...,24.0,0.7,0.5,14.0,73.0,1.3,0.633333,-0.300000,2.466667,-0.133333
1,2017-10-28,1.0,Home,1.0,2.0,0.8,1.1,45.0,18.0,7.0,...,26.0,1.1,0.8,7.0,55.0,0.9,0.533333,-0.200000,1.900000,0.433333
2,2017-11-04,2.0,Home,1.0,0.0,1.2,0.8,35.0,17.0,5.0,...,10.0,0.8,1.2,11.0,65.0,-0.8,0.666667,0.000000,1.633333,0.366667
3,2017-11-25,1.0,Home,1.0,2.0,0.4,0.7,44.0,6.0,2.0,...,8.0,0.7,0.4,12.0,56.0,1.3,0.833333,-0.166667,0.866667,0.466667
4,2017-12-08,2.0,Home,2.0,0.0,3.4,0.3,36.0,23.0,9.0,...,16.0,0.3,3.4,6.0,64.0,-0.3,0.800000,0.200000,0.866667,0.466667


In [18]:
df = matches_rolling
df = df.sort_values('date')

In [19]:
df.head()

,date,result,venue,gf,ga,xg,xga,poss,sh,sot,...,opp_c,xg_opp,xga_opp,sh_opp,poss_opp,g-xg_opp,xgR,g-xgR,xg_oppR,g-xg_oppR
1167,2017-09-21,2.0,Home,3.0,0.0,1.6,1.0,33.0,12.0,8.0,...,24.0,1.0,1.6,14.0,67.0,-1.0,1.466667,-0.133333,1.100000,-0.100000
959,2017-09-23,1.0,Home,0.0,3.0,1.1,1.3,40.0,14.0,2.0,...,4.0,1.3,1.1,10.0,60.0,-0.3,1.866667,-0.866667,0.833333,0.166667
518,2017-09-29,0.0,Home,3.0,3.0,2.4,1.7,62.0,16.0,9.0,...,12.0,1.7,2.4,11.0,38.0,1.3,1.500000,-0.166667,1.266667,0.066667
1168,2017-09-30,1.0,Home,0.0,2.0,1.0,2.3,63.0,13.0,0.0,...,0.0,2.3,1.0,8.0,37.0,-0.3,1.566667,0.433333,1.333333,-0.333333
625,2017-09-30,2.0,Home,2.0,1.0,1.8,1.5,61.0,16.0,9.0,...,11.0,1.5,1.8,10.0,39.0,-0.5,1.800000,-0.800000,1.533333,0.800000


## Now let's set up our model, train, test and see the results.

In [20]:
#Setting up our neural network model with 4 input layers, 8 hidden layers and 3 output layers (Win, Draw, Lose). 

model = keras.Sequential([
    keras.layers.Flatten(input_shape=(4,)),
    keras.layers.Dense(8, activation="relu"),
    keras.layers.Dense(3, activation="softmax")
])

In [21]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [22]:
#Setting up training and testing slices.
train = df[df['date'] <= '2021-01-01']
test = df[df['date'] > '2021-01-01']

In [30]:
model.fit(train[new_cols], train['result'], epochs=13)

Epoch 1/13
38/38 [==============================] - 0s 1ms/step - loss: 1.0574 - accuracy: 0.4516
Epoch 2/13
38/38 [==============================] - 0s 1ms/step - loss: 1.0565 - accuracy: 0.4549
Epoch 3/13
38/38 [==============================] - 0s 1ms/step - loss: 1.0559 - accuracy: 0.4558
Epoch 4/13
38/38 [==============================] - 0s 1ms/step - loss: 1.0548 - accuracy: 0.4549
Epoch 5/13
38/38 [==============================] - 0s 1ms/step - loss: 1.0542 - accuracy: 0.4599
Epoch 6/13
38/38 [==============================] - 0s 1ms/step - loss: 1.0535 - accuracy: 0.4599
Epoch 7/13
38/38 [==============================] - 0s 1ms/step - loss: 1.0528 - accuracy: 0.4608
Epoch 8/13
38/38 [==============================] - 0s 1ms/step - loss: 1.0523 - accuracy: 0.4599
Epoch 9/13
38/38 [==============================] - 0s 1ms/step - loss: 1.0515 - accuracy: 0.4641
Epoch 10/13
38/38 [==============================] - 0s 1ms/step - loss: 1.0511 - accuracy: 0.4649
Epoch 11/13
38/38 [

In [24]:
test_loss, test_acc = model.evaluate(test[new_cols], test['result'])

27/27 [==============================] - 0s 1ms/step - loss: 1.0500 - accuracy: 0.4548


In [25]:
prediction = model.predict(test[new_cols])

27/27 [==============================] - 0s 1ms/step


In [26]:
prediction[:5]     #Draw, Away, Home   <- predictions are in that order.

array([[0.306652  , 0.28858393, 0.4047641 ],
       [0.26090324, 0.26451614, 0.47458062],
       [0.3197328 , 0.2914878 , 0.38877946],
       [0.21976328, 0.26128116, 0.5189555 ],
       [0.2685617 , 0.28841123, 0.44302708]], dtype=float32)

In [27]:
#Adding the probabilities to our sweet DataFrame.
test['home'] = prediction[:,2]
test['draw'] = prediction[:,0]
test['away'] = prediction[:,1]

C:\Users\PC\AppData\Local\Temp\ipykernel_7308\127324621.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['home'] = prediction[:,2]
C:\Users\PC\AppData\Local\Temp\ipykernel_7308\127324621.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['draw'] = prediction[:,0]
C:\Users\PC\AppData\Local\Temp\ipykernel_7308\127324621.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the 

In [28]:
test = test[['date', 'team', 'opponent', 'home', 'draw', 'away']].round(2)

In [29]:
#Now we have the probabilities that our model is set for each even to happen in a certain match.
test

,date,team,opponent,home,draw,away
471,2021-01-02,Betis,Sevilla,0.40,0.31,0.29
1504,2021-01-02,Real Madrid,Celta Vigo,0.47,0.26,0.26
914,2021-01-02,Getafe,Valladolid,0.39,0.32,0.29
2004,2021-01-02,Villarreal,Levante,0.52,0.22,0.26
702,2021-01-03,Eibar,Granada,0.44,0.27,0.29
...,...,...,...,...,...,...
1550,2023-06-04,Real Madrid,Athletic Club,0.56,0.21,0.23
517,2023-06-04,Betis,Valencia,0.43,0.28,0.29
1941,2023-06-04,Valladolid,Getafe,0.22,0.41,0.38
1323,2023-06-04,Mallorca,Rayo Vallecano,0.30,0.37,0.33
